# **Entrainement du modèle de détection des batiments détruits à partir du modèle Pré-entrainé sur la base euroSAT**

In [1]:
import os
import time
import copy

import tensorflow as tf
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1050 Ti


In [3]:
data_transforms = {
    'train': transforms.Compose([
        #transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        #transforms.Resize(224),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../data/images/ezzor'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

Chargement du modèle pré-trained sur euroSAT

In [4]:
model = models.resnet50(pretrained=False, progress=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10) # Ici le nombre de layer final correspond au nombre de label danbs la abse euroSAT
model.load_state_dict(torch.load('../euroSAT/euroSAT_dataset/models/euroSAT_model_98_stat_dict.pt'))

<All keys matched successfully>

Modification du Fully Connected layer final pour le faire correspondre au nombre de label de notre base (0/1)

In [5]:
model.fc = nn.Linear(num_ftrs, 2)

### **Phase d'apprentissage sur la base Ezzor**

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

La phase d'apprentissage s'effectue sur l'ensemble des paramètres du modèle  
*A essayer* : Seulement considérer la dernière couche fc lors de l'optimisation pour comparer les résultats

In [8]:
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# On optimise sur tout les paramètre
optimizer_ft = optim.Adam(model.parameters())
# model_ft.fc.parameters()

# Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)
exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1,
                                           patience=3, verbose=True, threshold=0.01, threshold_mode='rel',
                                           cooldown=0, min_lr=0, eps=1e-08)

In [10]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.4710 Acc: 0.7696
test Loss: 0.4570 Acc: 0.8033

Epoch 1/9
----------
train Loss: 0.4665 Acc: 0.7816
test Loss: 0.4491 Acc: 0.8053

Epoch 2/9
----------
train Loss: 0.4724 Acc: 0.7834
test Loss: 0.4575 Acc: 0.8012

Epoch 3/9
----------
train Loss: 0.4698 Acc: 0.7751
test Loss: 0.4577 Acc: 0.7971

Epoch 4/9
----------
train Loss: 0.4644 Acc: 0.7880
test Loss: 0.4585 Acc: 0.7992

Epoch 5/9
----------
train Loss: 0.4660 Acc: 0.7917
test Loss: 0.4561 Acc: 0.7930

Epoch 6/9
----------
train Loss: 0.4634 Acc: 0.7926
test Loss: 0.4567 Acc: 0.8033

Epoch 7/9
----------
train Loss: 0.4745 Acc: 0.7843
test Loss: 0.4666 Acc: 0.7848

Epoch 8/9
----------
train Loss: 0.4538 Acc: 0.7908
test Loss: 0.4547 Acc: 0.8033

Epoch 9/9
----------
train Loss: 0.4603 Acc: 0.7926
test Loss: 0.4577 Acc: 0.8012

Training complete in 34m 49s
Best test Acc: 0.805328


In [12]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 